# Graph Structure Analysis

Before training any GNN, let's actually look at what the graph looks like. The idea is that fraud nodes might have different structural properties (higher degree, different centrality, etc.) that the GNN can pick up on.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from src.data.dataset import create_synthetic_fraud_data
from src.data.graph_builder import TransactionGraphBuilder

%matplotlib inline

In [ ]:
# generate synthetic transactions
df = create_synthetic_fraud_data(
    num_users=1000,
    num_merchants=200,
    num_transactions=10000,
    fraud_rate=0.05,
)

# build the PyG graph
builder = TransactionGraphBuilder()
pyg_data = builder.build_graph(df)

print(f'Nodes: {pyg_data.num_nodes} ({pyg_data.num_users} users + {pyg_data.num_merchants} merchants)')
print(f'Edges: {pyg_data.edge_index.shape[1]} (bidirectional, so {pyg_data.edge_index.shape[1] // 2} unique)')
print(f'Node feature dim: {pyg_data.x.shape[1]}')
print(f'Fraud rate: {df["is_fraud"].mean():.2%}')

## Build a NetworkX graph for analysis

PyG is great for training but networkx is way easier for structural analysis. Let's build a bipartite graph (users <-> merchants) and tag edges with fraud labels.

In [ ]:
G = nx.Graph()

users = df['user_id'].unique()
merchants = df['merchant_id'].unique()

G.add_nodes_from(users, bipartite=0, node_type='user')
G.add_nodes_from(merchants, bipartite=1, node_type='merchant')

# add edges with fraud attribute
for _, row in df.iterrows():
    if G.has_edge(row['user_id'], row['merchant_id']):
        # multiple transactions between same user-merchant, keep max fraud
        G[row['user_id']][row['merchant_id']]['fraud'] = max(
            G[row['user_id']][row['merchant_id']]['fraud'], row['is_fraud']
        )
    else:
        G.add_edge(row['user_id'], row['merchant_id'], fraud=row['is_fraud'])

print(f'NetworkX graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges')
print(f'Connected components: {nx.number_connected_components(G)}')

## Subgraph visualization

Plotting the whole graph would be a mess with 1200 nodes. Let's grab a small subgraph around some fraud nodes and see if we can spot any patterns visually.

In [ ]:
# get users involved in fraud
fraud_users = df[df['is_fraud'] == 1]['user_id'].unique()[:15]

# grab their 1-hop neighborhood
subgraph_nodes = set(fraud_users)
for u in fraud_users:
    subgraph_nodes.update(G.neighbors(u))

# also add some legit users for contrast
legit_users = df[df['is_fraud'] == 0]['user_id'].unique()[:20]
for u in legit_users:
    if u in G:
        subgraph_nodes.add(u)
        subgraph_nodes.update(list(G.neighbors(u))[:3])  # just a few neighbors

sub = G.subgraph(subgraph_nodes)
print(f'Subgraph: {sub.number_of_nodes()} nodes, {sub.number_of_edges()} edges')

# color nodes
node_colors = []
for n in sub.nodes():
    if G.nodes[n].get('node_type') == 'merchant':
        node_colors.append('#999999')  # gray for merchants
    elif n in fraud_users:
        node_colors.append('#e74c3c')  # red for fraud users
    else:
        node_colors.append('#3498db')  # blue for legit users

fig, ax = plt.subplots(figsize=(12, 8))
pos = nx.spring_layout(sub, seed=42, k=0.5)
nx.draw_networkx(
    sub, pos, ax=ax,
    node_color=node_colors,
    node_size=40,
    with_labels=False,
    edge_color='#cccccc',
    alpha=0.8,
    width=0.5,
)
ax.set_title('Subgraph: red=fraud users, blue=legit users, gray=merchants')
plt.tight_layout()

## Degree distribution

In fraud detection papers, fraudulent nodes sometimes show unusual connectivity patterns. Let's check.

In [ ]:
user_degrees = {u: G.degree(u) for u in users}
merchant_degrees = {m: G.degree(m) for m in merchants}

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(list(user_degrees.values()), bins=30, color='steelblue', alpha=0.7)
axes[0].set_xlabel('Degree')
axes[0].set_ylabel('Count')
axes[0].set_title('User Degree Distribution')

axes[1].hist(list(merchant_degrees.values()), bins=30, color='coral', alpha=0.7)
axes[1].set_xlabel('Degree')
axes[1].set_ylabel('Count')
axes[1].set_title('Merchant Degree Distribution')

plt.tight_layout()

## Fraud vs legit node degree

The real question -- do users involved in fraud have different degree distributions?

In [ ]:
# tag users by whether they've been involved in any fraud
all_fraud_users = set(df[df['is_fraud'] == 1]['user_id'].unique())

fraud_degrees = [user_degrees[u] for u in users if u in all_fraud_users]
legit_degrees = [user_degrees[u] for u in users if u not in all_fraud_users]

print(f'Fraud users: {len(fraud_degrees)}, avg degree: {np.mean(fraud_degrees):.1f}')
print(f'Legit users: {len(legit_degrees)}, avg degree: {np.mean(legit_degrees):.1f}')

fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(legit_degrees, bins=20, alpha=0.6, label='Legit', color='steelblue', density=True)
ax.hist(fraud_degrees, bins=20, alpha=0.6, label='Fraud', color='indianred', density=True)
ax.set_xlabel('Degree (unique merchants)')
ax.set_ylabel('Density')
ax.set_title('User Degree: Fraud vs Legit')
ax.legend()
plt.tight_layout()

Interesting -- fraud users tend to have slightly higher degree on average. Makes sense because in the synthetic data, fraud transactions are spread across users randomly so users with more transactions have a higher chance of being tagged. In real data the pattern might be different (e.g., fraud rings using many accounts).

## Node feature distributions

Let's look at the computed node features from the graph builder and see if fraud-adjacent nodes look different.

In [ ]:
# the node features are stored in pyg_data.x
# first num_users rows are users, rest are merchants
feature_names = [
    'tx_count', 'avg_amount', 'std_amount', 'max_amount',
    'unique_merchants', 'merch_tx_count', 'merch_avg_amount', 'merch_unique_users'
]

user_feats = pyg_data.x[:pyg_data.num_users].numpy()

# map fraud flag to users
user_list = df['user_id'].unique()  # same order as builder.user_mapping
is_fraud_user = np.array([1 if u in all_fraud_users else 0 for u in user_list])

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for idx, ax in enumerate(axes.flat):
    if idx >= 4:
        break
    feat = user_feats[:, idx]
    ax.hist(feat[is_fraud_user == 0], bins=25, alpha=0.6, label='Legit', color='steelblue', density=True)
    ax.hist(feat[is_fraud_user == 1], bins=25, alpha=0.6, label='Fraud', color='indianred', density=True)
    ax.set_title(feature_names[idx])
    ax.legend(fontsize=8)

plt.suptitle('User Node Features: Fraud vs Legit', y=1.02)
plt.tight_layout()

The `max_amount` and `avg_amount` features clearly separate fraud users -- that's expected since we multiply fraud amounts by 3x in the synthetic generator. Good to confirm the graph builder is capturing this.

## Centrality analysis

Let's check a few centrality metrics. This takes a bit on the full graph so we'll use the user projection.

In [ ]:
# project to user-only graph (two users connected if they share a merchant)
user_proj = nx.bipartite.projected_graph(G, users)
print(f'User projection: {user_proj.number_of_nodes()} nodes, {user_proj.number_of_edges()} edges')

# degree centrality
deg_cent = nx.degree_centrality(user_proj)

# clustering coefficient
clustering = nx.clustering(user_proj)

# betweenness is slow on big graphs, sample it
betweenness = nx.betweenness_centrality(user_proj, k=100, seed=42)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

metrics = [
    ('Degree Centrality', deg_cent),
    ('Clustering Coeff', clustering),
    ('Betweenness Centrality', betweenness),
]

for ax, (name, metric) in zip(axes, metrics):
    fraud_vals = [metric.get(u, 0) for u in users if u in all_fraud_users]
    legit_vals = [metric.get(u, 0) for u in users if u not in all_fraud_users]

    ax.hist(legit_vals, bins=25, alpha=0.6, label='Legit', color='steelblue', density=True)
    ax.hist(fraud_vals, bins=25, alpha=0.6, label='Fraud', color='indianred', density=True)
    ax.set_title(name)
    ax.legend(fontsize=8)

plt.tight_layout()

In [ ]:
# quick summary stats
print('=== Degree Centrality ===')
print(f'  Fraud mean: {np.mean([deg_cent.get(u, 0) for u in users if u in all_fraud_users]):.4f}')
print(f'  Legit mean: {np.mean([deg_cent.get(u, 0) for u in users if u not in all_fraud_users]):.4f}')

print('\n=== Clustering Coefficient ===')
print(f'  Fraud mean: {np.mean([clustering.get(u, 0) for u in users if u in all_fraud_users]):.4f}')
print(f'  Legit mean: {np.mean([clustering.get(u, 0) for u in users if u not in all_fraud_users]):.4f}')

print('\n=== Betweenness Centrality ===')
print(f'  Fraud mean: {np.mean([betweenness.get(u, 0) for u in users if u in all_fraud_users]):.4f}')
print(f'  Legit mean: {np.mean([betweenness.get(u, 0) for u in users if u not in all_fraud_users]):.4f}')

The centrality differences aren't huge, which makes sense -- the synthetic data generates fraud uniformly at random. In real-world fraud you'd expect to see fraud rings (dense subgraphs) and higher betweenness for money mule accounts. The GNN should still be able to learn these structural signals combined with the node/edge features.

## Takeaways

- Graph is bipartite (users <-> merchants), pretty well connected since we have 10k transactions over 1200 nodes
- Fraud users have slightly higher degree and higher amount features (expected from synthetic generator)
- Centrality metrics show small differences -- the GNN will need to combine topology with features
- Next step: train GraphSAGE and see if message passing actually helps vs a flat MLP baseline